In [4]:
import os
import pandas as pd
import numpy as np
import datetime
import psycopg2

In [56]:
def get_db_connex():
    conn = psycopg2.connect(host="localhost", database="crosstown")
    return conn.cursor()

In [6]:
columns = [
    'AD',
    'ED',
    'County',
    'EDAD Status',
    'Event',
    'Party/Independent Body',
    'Office/Position Title',
    'District Key',
    'VoteFor',
    'Unit Name',
    'Tally'
]

In [59]:
def load_data(filepath, cols = list(range(11,22)), header = None):
    return pd.read_csv(
        filepath, 
        names = columns,
        usecols = cols,
        header = header
    )

def data_to_db(df):
    df.to_sql(name='results', con=conn, schema='public')
    
def data_from_db(cur, query):
    cur.execute(query)
    return cur.fetchall()
    
def write_metadata(bod_id, df):
    date, event_name = get_date_and_event_name(df)
    office = get_office(df)
    district = get_district(df)
    rows = len(df)
    sql_query = ("INSERT INTO datasets(bod_id, date, event_name, office, district_key, rows) VALUES ('"+
        bod_id+
        "', to_date('"+
        date+
        "'::text, 'YYYY-MM-DD'),'"+
        event_name + 
        "','" +
        office+
        "','"+
        str(district)+
        "',"+
        str(rows)+
        ");"
    )
    query = 'psql -h localhost crosstown -c "' + sql_query + '"'
    print('** meta query  **', query)
    res = os.system(query)
    print('** meta status ** ', res)

def write_data(bod_id, df):
    ## append bod_id column to end of DF
    df['bod_id'] = bod_id
    data_path = get_path(bod_id, df)
    full_path = './data/clean/'+data_path
    df.to_csv(full_path, index=False)
#     print('** skip write_data')
    query = ('cat '+
        full_path+
        ' | psql -h localhost crosstown -c '+
             '"COPY results(ad, ed, county, edad_status, event, party,'+
             ' office, district_key, vote_for, unit_name, tally, bod_id) from stdin CSV HEADER"'
    )
    print('** data query  ** ', query)    
    res = os.system(query)
    print('** data status ** ', res)
    
def get_path(bod_id, df):
    event = df['Event'][0].split(' - ')
    return event[1][-4:] + '/' + bod_id + '.csv'

def get_date_and_event_name(df):
    event = df['Event'][0].split(' - ')
    return (str(datetime.datetime.strptime(event[1][-10:], '%m/%d/%Y').date()), event[0])

def get_office(df):
    return df["Office/Position Title"][0]

def get_district(df):
    return df["District Key"][0]

def count_cols(path):
    data = pd.read_csv(path, nrows=1)
    return data.size

def control(year, raw_file):
    bod_id = raw_file[0:11]
    path = './data/raw/'+year+'/'+raw_file
    
    nCols = count_cols(path)
    
    if nCols == 22:
        cols = list(range(11,22))
        header = None
    elif nCols == 11:
        cols = list(range(0, 11))
        header = 0
    else:
        print("Unrecognized number of columns, exiting on ", bod_id)
        return

    data = load_data(path, cols, header)
    if len(data) == 0:
        print("No data, exiting on ", bod_id)
        return
    date = get_date(data)
    if is_dataset_in_datasets(bod_id, date):
        print ("Already loaded, exiting on ", bod_id)
        return
    write_data(bod_id, data)
    write_metadata(bod_id, data)
    
def is_dataset_in_datasets(bod_id, date):
    res = data_from_db(
        cur, 
        """SELECT * FROM datasets WHERE bod_id = '"""+ bod_id +"""' and date = '"""+ date +"""' """
    )
    return len(res) > 0

In [57]:
## https://vote.nyc/page/election-results-summary
## CAUTION ID bod_id is unique by office, not by election date
## datasets is unique by bod_id / date
cur = get_db_connex()
years = list(filter(lambda x: x != '.DS_Store', os.listdir('./data/raw')))
loaded_files_bod_ids = map(lambda x: (x[0], str(x[1])), data_from_db(cur, """SELECT * FROM datasets"""))
years

['2013', '2014', '2015', '2012', '2017', '2019', '2020', '2018', '2016']

In [60]:
year = '2020'
files = list(filter(lambda x: x != '.DS_Store', os.listdir('./data/raw/'+year)))
for f in files:
    control(year, f)
# raw_file = files[fileIndex]
# extraCols = False
# bod_id = raw_file[0:11]
    
# if extraCols:
#     cols = list(range(11,22))
#     header = None
# else:
#     cols = list(range(0, 11))
#     header = 0

# data = load_data('./data/raw/'+raw_file, cols, header)
# date = get_date(data)
# is_dataset_in_datasets(bod_id, date)

** data query  **  cat ./data/clean/2020/01202400085.csv | psql -h localhost crosstown -c "COPY results(ad, ed, county, edad_status, event, party, office, district_key, vote_for, unit_name, tally, bod_id) from stdin CSV HEADER"
** data status **  0
** meta query  ** psql -h localhost crosstown -c "INSERT INTO datasets(bod_id, date, event_name, office, district_key, rows) VALUES ('01202400085', to_date('2020-06-23'::text, 'YYYY-MM-DD'),'Primary Election 2020','Member of the Assembly','85',613);"
** meta status **  0
** data query  **  cat ./data/clean/2020/01402400031.csv | psql -h localhost crosstown -c "COPY results(ad, ed, county, edad_status, event, party, office, district_key, vote_for, unit_name, tally, bod_id) from stdin CSV HEADER"
** data status **  0
** meta query  ** psql -h localhost crosstown -c "INSERT INTO datasets(bod_id, date, event_name, office, district_key, rows) VALUES ('01402400031', to_date('2020-06-23'::text, 'YYYY-MM-DD'),'Primary Election 2020','Member of the A

In [48]:
len("Special Election City Council District 45 - 05/14/2019")

54

In [55]:
cur.close()